In [0]:
import json,os
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime, timedelta
import re
from pyspark.sql import Window

In [0]:
start_date= (datetime.now() + timedelta(days = -30)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2022-03-19
end_date: 2022-04-18

In [0]:
source = 'OSCE'
xml_veraion = '2.0'
START_DATE = '2022-01-01' 
END_DATE = '2022-02-01'

In [0]:
# df_raw = (spark.read.format('delta')
#           .load(f'/mnt/trendmicrobif/TBL_DATA_RAW')
#           .filter(F.col('source') == source)
#           .filter(F.col('xml_version') == xml_veraion )
#      )
# df_raw.createOrReplaceTempView('TBL_DATA_RAW')

# df_xpath = (spark.read.format('csv')
#          .option('header', True)
#          .load(f'/mnt/trendmicrobif/TBL_RAW_XPATH.csv')
#          .filter(F.col('source') == source)
#          .filter(F.col('xml_version') == xml_veraion)
         
#         )
# df_config = (spark.read.format('csv')
#          .option('header', True)
#          .load(f'/mnt/trendmicrobif/TBL_XML_RAW_CONFIG.csv')
#          .filter(F.col('source') == source)
#          .filter(F.col('xml_version') == xml_veraion)
         
#         )

In [0]:
#synapse 
spark.conf.set('fs.azure.account.key.gdwpocdls.dfs.core.windows.net', dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'StorageAccount-gdwpocdls-AccessKey'));
_url = dbutils.secrets.get(scope = 'BIFC-asagdwpocws-etl', key = 'Synapse-asagdwpocetl-ConnectionString');


In [0]:
df_raw = (spark.read
          .format('com.databricks.spark.sqldw')
          .option('url', _url)
          .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
          .option('useAzureMSI', 'true')
          .option('dbTable', 'BIFC.TBL_DATA_RAW')
          .load()
          .filter(F.col('file_date') > START_DATE)
          .filter(F.col('file_date') < END_DATE)
          .filter(F.col('source') == source)
          .filter(F.col('xml_version') == xml_veraion )
          )
df_raw.createOrReplaceTempView('TBL_DATA_RAW')

df_xpath = (spark.read
                    .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC.TBL_RAW_XPATH')
                    .load()
                    .filter(F.col('source') == source)
                    .filter(F.col('xml_version') == xml_veraion)
                   )
display(df_xpath)
df_raw_config = (spark.read
                    .format('com.databricks.spark.sqldw')
                    .option('url', _url)
                    .option('tempDir', 'abfss://bifconnect@gdwpocdls.dfs.core.windows.net/tempDirs')
                    .option('useAzureMSI', 'true')
                    .option('dbTable', 'BIFC_ETL.TBL_XML_RAW_CONFIG')
                    .load()
                    .filter(F.col('source') == source)
         .filter(F.col('xml_version') == xml_veraion)
                   )
display(df_raw_config)

ID,source,xml_version,xpath,create_date,latest_feedback_date,file_count
8252,OSCE,2.0,/Data/Products/Product/Server/AskRebootHistory/AskRebootHistory_Config/DaysToKeepLogs,2022-03-05,null,null
8236,OSCE,2.0,/Data/Products/Product/Server/AskRebootHistory/Summary/Item/@ByDay,2022-02-26,null,null
8212,OSCE,2.0,/Data/Products/Product/EndPoint/MITRE_CFG/MITRE_MODE/@desktop_disable,2022-01-05,null,null
8204,OSCE,2.0,/Data/Products/Product/Server/ProxySettings/AgentProxy/AgentInternalProxy/AutoProxy/Proxy_Pwd_Set,2021-11-30,null,null
8188,OSCE,2.0,/Data/Products/Product/Server/ProxySettings/AgentProxy/AgentExternalProxy/URLFilterProxyAccount,2021-11-09,null,null
8180,OSCE,2.0,/Data/Products/Product/Server/ProxySettings/AgentProxy/AgentInternalProxy/WindowsProxy/Proxy_Login,2021-11-09,null,null
8172,OSCE,2.0,/Data/Products/Product/Server/ProxySettings/ServerProxy/Internet_Proxy_Socks,2021-11-05,null,null
8156,OSCE,2.0,/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteMinute,2021-11-03,null,null
8148,OSCE,2.0,/Data/Products/Product/Server/AgentGlobalSetting/ScheduleUpdateStartHour,2021-11-03,null,null
8140,OSCE,2.0,/Data/Products/Product/Server/FeatureFlagHiddenKey/AllowAMSIBypassTrickNonBlocking,2021-11-02,null,null


xpath_id,source,xml_version,level,xpath,raw_group_root,group_root,master_detail,default_column_name,default_data_type,default_data_length,sample_value,group_level
4989,OSCE,2.0,6,/Data/Products/Product/EndPoint/Firewall/@server_on,/Data/Products/Product/EndPoint/Firewall,/Data/Products/Product/EndPoint/Firewall,M,Firewall_server_on,int,null,0,5
4997,OSCE,2.0,8,/Data/Products/Product/EndPoint/WebReputation/External/Item/@Name,/Data/Products/Product/EndPoint/WebReputation/External/Item,/Data/Products/Product/EndPoint/WebReputation/External/Item,D,External_Item_Name,varchar,53,SendQuerySmartProtectionServer,7
5005,OSCE,2.0,9,/Data/Products/Product/EndPoint/WebReputation/External/SecurityLevel/medium/@server,/Data/Products/Product/EndPoint/WebReputation/External/SecurityLevel/medium,/Data/Products/Product/EndPoint/WebReputation/External/SecurityLevel,M,medium_server,int,null,31,8
5013,OSCE,2.0,9,/Data/Products/Product/EndPoint/WebReputation/Internal/SecurityLevel/low/@desktop,/Data/Products/Product/EndPoint/WebReputation/Internal/SecurityLevel/low,/Data/Products/Product/EndPoint/WebReputation/Internal/SecurityLevel,M,SecurityLevel_low_desktop,int,null,164,8
5021,OSCE,2.0,7,/Data/Products/Product/Server/BT_SuspiciousObjectList/BT_USE_DNS/@desktop_off,/Data/Products/Product/Server/BT_SuspiciousObjectList/BT_USE_DNS,/Data/Products/Product/Server/BT_SuspiciousObjectList,M,BT_USE_DNS_desktop_off,int,null,2,6
5029,OSCE,2.0,7,/Data/Products/Product/Server/BT_SuspiciousObjectList/BT_USE_IP/@desktop_off,/Data/Products/Product/Server/BT_SuspiciousObjectList/BT_USE_IP,/Data/Products/Product/Server/BT_SuspiciousObjectList,M,BT_USE_IP_desktop_off,int,null,2,6
5037,OSCE,2.0,7,/Data/Products/Product/Server/BT_SuspiciousObjectList/SUSP_OBJ_LIST_USE_CM/@desktop_off,/Data/Products/Product/Server/BT_SuspiciousObjectList/SUSP_OBJ_LIST_USE_CM,/Data/Products/Product/Server/BT_SuspiciousObjectList,M,SUSP_OBJ_LIST_USE_CM_desktop_off,int,null,2,6
5222,OSCE,2.0,5,/Data/Products/Product/Server/CLPID,/Data/Products/Product/Server,/Data/Products/Product/Server,M,Server_CLPID,varchar,63,00c93bd2-23e9-4296-873c-c8d7ec92b79b,4
5230,OSCE,2.0,5,/Data/Products/Product/xdr_onboarding/xdr_xbc_uninstall,/Data/Products/Product/xdr_onboarding,/Data/Products/Product/xdr_onboarding,M,xdr_onboarding_xdr_xbc_uninstall,int,null,-1,4
99,OSCE,2.0,7,/Data/Products/Product/EndPoint/AgentGlobalSettingInClient/Item/@desktop_on,/Data/Products/Product/EndPoint/AgentGlobalSettingInClient/Item,/Data/Products/Product/EndPoint/AgentGlobalSettingInClient/Item,D,Item_desktop_on,int,null,2,6


In [0]:
#get config
df_xpath_config = (df_xpath.alias('r').join(df_config.alias('c'), [F.col('r.source')==F.col('c.source'),
                                                                F.col('r.xml_version')==F.col('c.xml_version'),
                                                                F.col('r.xpath')==F.col('c.xpath')
                                                               ], 'left')
                  .withColumn('level', F.coalesce('c.level',F.lit('0')))
                  .withColumn('raw_group_root', F.coalesce('c.raw_group_root','r.xpath'))
                  .withColumn('group_root', F.coalesce('c.group_root','r.xpath'))
                  .withColumn('master_detail', F.coalesce('c.master_detail',F.lit('M')))
                  .withColumn('column_name', F.coalesce('c.default_column_name','r.xpath'))
                  .withColumn('data_type', F.coalesce('c.default_data_type',F.lit('varchar')))
                  .withColumn('data_length', F.coalesce('c.default_data_length',F.lit('1000')))
                  .withColumn('sample_value', F.coalesce('c.sample_value',F.lit('')))
                  .withColumn('ID', F.row_number().over(Window.orderBy(F.col('r.source'),F.col('r.xml_version'),F.col('r.xpath'))))
                  .select('ID', 'r.source', 'r.xml_version', 'level', 'r.xpath', 'raw_group_root', 'group_root', 'master_detail', 'column_name', 'data_type', 'data_length', 'sample_value')
                  )
display(df_xpath_config)  

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1310704823771877> in <module> 
 1 #get config 
 ----> 2 df_xpath_config = (df_xpath.alias('r').join(df_config.alias('c'), [F.col('r.source')==F.col('c.source'),
 3 F . col ( 'r.xml_version' ) == F . col ( 'c.xml_version' ) , 
 4 F . col ( 'r.xpath' ) == F . col ( 'c.xpath' ) 
 5 ], 'left')

 NameError : name 'df_config' is not defined

In [0]:
data = [('test_data','<?xml version="1.0" encoding="utf-8"?><data> <product_info><hardware_info><cpu>Intel(R) Xeon(R) Gold 5220 CPU @ 2.20GHz x 1 core(s)</cpu><cpu>Intel(R) Xeon(R) Gold 5220 CPU @ 2.20GHz x 1 core(s)</cpu><hdd_partitions><filestores><avail>175981248</avail><size>197460232</size></filestores><varlog><avail>64712540</avail><size>103212320</size></varlog></hdd_partitions><hdd_size>314572800</hdd_size></hardware_info></product_info></data>')]
df = spark.createDataFrame(data,["file_name","file_content"])
df.createOrReplaceTempView('testdata')


In [0]:
%sql 
select file_name, xpath(file_content,'/data/product_info/hardware_info/cpu/text()') as value from testdata

In [0]:
%sql 
select file_name, xpath(file_content,'/Data/Products/Product/EndPoint/MITRE_CFG/MITRE_MODE/@desktop_disable') as value from TBL_DATA_RAW

In [0]:
xpath_full = "/Data/Products/Product/EndPoint/MITRE_CFG/MITRE_MODE/@desktop_detection"
sql_script = f"select file_name, xpath(file_content,'{xpath_full}') as value from TBL_DATA_RAW"
xpath_tmp = (spark
   .sql(sql_script)
   .select('file_name', list_to_lenght_udf('value').alias('count'))
   .filter(F.col('count')>1)
)

display(xpath_tmp)

In [0]:
def list_to_lenght(list): 
    return len(list)
list_to_lenght_udf = udf(list_to_lenght, IntegerType())

In [0]:
# update master_detail, raw_group_root, group_root, level, column_name
update_set = []
for row in df_xpath_config.collect():
        xpath = row.xpath #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/@ByCount
        xpath_list = xpath.split('/')
        xpath_full_text = '/'.join(xpath_list[:-1])+'/'+xpath_list[-1].replace('@','') +'/text()' #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/ByCount/text()
        xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','')  #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/@ByCount
        group_root = '/'.join(xpath_list[:-2])+'/'+xpath_list[-2] #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item


        update_dict = {}
        if(row.level == '0'):
            master_detail = 'M'
            sql_script = f"select file_id, xpath(file_content,'{xpath_full}') as value from TBL_DATA_RAW"
            xpath_tmp = (spark
                       .sql(sql_script)
                       .select('file_id', list_to_lenght_udf('value').alias('count'))
                       .filter(F.col('count')>1)
                       )
            if(xpath_tmp.count() > 0): 
                master_detail = 'D'
                update_dict['ID'] = row.ID
                update_dict['xpath'] = xpath
                update_dict['xpath_fix'] = xpath_full
                update_dict['raw_group_root'] = group_root
                update_dict['master_detail'] = master_detail
                update_set.append(update_dict)
            else:
                sql_script = f"select file_id, xpath(file_content,'{xpath_full_text}') as value from TBL_DATA_RAW"
                xpath_tmp = (spark
                             .sql(sql_script)
                             .select('file_id', list_to_lenght_udf('value').alias('count'))
                             .filter(F.col('count')>1)
                             )
                if(xpath_tmp.count() > 0): 
                    master_detail = 'D'
                    update_dict['ID'] = row.ID
                    update_dict['xpath'] = xpath
                    update_dict['raw_group_root'] = xpath
                    update_dict['xpath_fix'] = xpath_full_text
                    update_dict['master_detail'] = master_detail

            update_set.append(update_dict)


In [0]:
def xpath_to_level(xpath): 
  return len(xpath.split('/'))-1

xpath_to_level_udf = udf(xpath_to_level, IntegerType())

def xpath_to_columnname(xpath): 
  xpath_list = xpath.split('/')
  return xpath_list[-1].replace('@','')

xpath_to_columnname_udf = udf(xpath_to_columnname, StringType())


update_rdd= spark.sparkContext.parallelize(update_set)
update_df = (spark
             .createDataFrame(update_rdd)
             .withColumn('level',xpath_to_level_udf('xpath'))
             .withColumn('column_name',xpath_to_columnname_udf('xpath'))
            )


display(update_df)

In [0]:
df_xpath_config_1 = (df_xpath_config.alias('con').join(update_df.alias('update'), F.col('update.ID')==F.col('con.ID'),'left')
             .withColumn('level1',F.when(F.col('update.level').isNull(), F.col('con.level')).otherwise(F.col('update.level')))
             .withColumn('master_detail1', F.when(F.col('update.master_detail').isNull(), F.col('con.master_detail')).otherwise(F.col('update.master_detail')))
             .withColumn('column_name1',F.when(F.col('update.column_name').isNull(), F.col('con.column_name')).otherwise(F.col('update.column_name')))
             .withColumn('raw_group_root1', F.when(F.col('update.raw_group_root').isNull(), F.col('con.raw_group_root')).otherwise(F.col('update.raw_group_root')))
             .withColumn('group_root1', F.col('raw_group_root1'))
             .select('con.ID', 'con.source', 'con.xml_version', 'level1', 'con.xpath', 'raw_group_root1', 'group_root1', 'master_detail1', 'column_name1', 'con.data_type', 'con.data_length', 'con.sample_value')
              .withColumnRenamed('master_detail1', 'master_detail')
              .withColumnRenamed('raw_group_root1', 'raw_group_root')
              .withColumnRenamed('level1', 'level')
              .withColumnRenamed('group_root1', 'group_root')
              .withColumnRenamed('column_name1', 'column_name')
  )

display(df_xpath_config_1)
#         .filter(F.col('ID')=='399'))

In [0]:
display(df_xpath_config.filter(F.col('ID')=='299'))
display(df_xpath_config_1.filter(F.col('ID')=='299'))

In [0]:
# update sample_data , data_type ,data_length

update_set = []
for row in df_xpath_config_1.collect():
    xpath = row.xpath #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/@ByCount
    xpath_list = xpath.split('/')
    xpath_full_text = '/'.join(xpath_list[:-1])+'/'+xpath_list[-1].replace('@','') +'/text()' #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/ByCount/text()
    xpath_full = '/'.join(xpath_list[:-1])+'/@'+xpath_list[-1].replace('@','')  #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/@ByCount
    group_root = '/'.join(xpath_list[:-2])+'/'+xpath_list[-2] #/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item

    update_dict = {}
    if(not row.sample_value):
    
      sql_script = f"select file_id, xpath(file_content,'{xpath_full}') as value from TBL_DATA_RAW"
      xpath_tmp = (spark
                   .sql(sql_script)
                   .select('file_id', 'value',list_to_lenght_udf('value').alias('count'))
                   .filter(F.col('count')>1)
                   .select('file_id', F.explode('value').alias('value'))
                   )
      

      if(xpath_tmp.count() > 0): 
        update_dict['ID'] = row.ID
        update_dict['xpath'] = xpath
        update_dict['sample_value'] = xpath_tmp.rdd.collect()[0][1] 
        update_set.append(update_dict)
    else:
      sql_script = f"select file_id, xpath(file_content,'{xpath_full_text}') as value from TBL_DATA_RAW"
      xpath_tmp = (spark
                   .sql(sql_script)
                   .select('file_id', 'value',list_to_lenght_udf('value').alias('count'))
                   .filter(F.col('count')>1)
                   .select('file_id', F.explode('value').alias('value'))
                   )
      if(xpath_tmp.count() > 0): 
        master_detail = 'D'
        update_dict['ID'] = row.ID
        update_dict['xpath'] = xpath
        update_dict['sample_value'] = xpath_tmp.rdd.collect()[0][1] 
        update_set.append(update_dict)
        

In [0]:
def is_number(num):
  pattern = re.compile(r'^[-+]?[-0-9]\d*\.\d*|[-+]?\.?[0-9]\d*$')
  result = pattern.match(num)
  if result:
    return True
  else:
    return False
  
def is_valid_data(str):
  try:
    if ":" in str:
      time.strptimr(str,"%Y-%m-%d %H:%M%:S")
    else:
      time.strptimr(str,"%Y-%m-%d")
    return True
  except:
    return False
  
data_schema = StructType([
    StructField('data_type', StringType(), True),
    StructField('data_length', StringType(), True)
])

def setDatatype(value):
  data_type = ''
  data_length = ''
  data_info = {}
  if(len(value)>8 and is_valid_data(value)):
      data_type = "datetime"
      data_length = ""
  elif(is_number(value)):
    
    whole_part =  len(str(value).split(".")[0])
    decimal_part = 0
    if (len(str(value).split(".")) == 2) :     
      decimal_part = len(str(value).split(".")[1])
      
    if(decimal_part == 0 ):
      data_type = "int"
      data_length = ""
    else:
      datatype = "numeric"
      data_length = f"{str(whole_part+decimal_part)},{decimal_part}"
  else:
    data_type = "varchar"    
    data_length = len(value)
    
  data_info['data_type'] = data_type
  data_info['data_length'] = data_length                     
  return data_info
                     
setDatatype_udf = udf(setDatatype,data_schema)

In [0]:
update_rdd= spark.sparkContext.parallelize(update_set)
update_df = (spark
             .createDataFrame(update_rdd)
             .withColumn('data_info', setDatatype_udf('sample_value'))
             .select('ID','xpath','sample_value','data_info.*')
            )


display(update_df)

In [0]:
df_xpath_config_2 = (df_xpath_config_1.alias('con').join(update_df.alias('update'), F.col('update.ID')==F.col('con.ID'),'left')
             .withColumn('sample_value1',F.when(F.col('update.sample_value').isNull(), F.col('con.sample_value')).otherwise(F.col('update.sample_value')))
             .withColumn('data_type1', F.when(F.col('update.data_type').isNull(), F.col('con.data_type')).otherwise(F.col('update.data_type')))
             .withColumn('data_length1',F.when(F.col('update.data_length').isNull(), F.col('con.data_length')).otherwise(F.col('update.data_length')))
             .select('con.ID', 'con.source', 'con.xml_version', 'con.level', 'con.xpath', 'con.raw_group_root', 'con.group_root', 'con.master_detail', 'con.column_name', 'data_type1', 'data_length1', 'sample_value1')
              .withColumnRenamed('data_type1', 'data_type')
              .withColumnRenamed('data_length1', 'data_length')
              .withColumnRenamed('sample_value1', 'sample_value')

  )

display(df_xpath_config_2)
#         .filter(F.col('ID')=='299'))

In [0]:
display(df_xpath_config.filter(F.col('ID')=='299'))
display(df_xpath_config_1.filter(F.col('ID')=='299'))
display(df_xpath_config_2.filter(F.col('ID')=='299'))

In [0]:
display(df_xpath_config_2)
df_xpath_config_2.write.saveAsTable('TBL_XML_RAW_CONFIG', mode='overwrite')

In [0]:
# sql_script = "select b.file_id, xpath(b.file_content,'/Data/Products/Product/Server/LogMaintenance/ScheduleDelete/ScheduledDeleteItems/Item/@ByCount') as value from TBL_DATA_RAW b"
# xpath_tmp = (spark
#                .sql(sql_script)
#                .select('file_id', 'value',list_to_lenght_udf('value').alias('count'))
#                .filter(F.col('count')>1)
#                .select('file_id', F.explode('value').alias('sample_value')) # explode list to rows  / select.* structure data to columns
# #                .withColumn('data_info', setDatatype_udf('sample_value'))
# #                .select('file_id','sample_value','data_info.*')
#              )

# display(xpath_tmp)